# Johdatus aihemallinnukseen

## Data
- Vauva.fi on suosituin suomalaisen aikakauslehden keskustelupalsta
- 23220 keskusteluketjua, joissa yhteensä 252738 viestiä Aihe vapaa -palstalta
- Aineiston keräys https://github.com/alkukampela/vauvascrape

## Kirjastot
- [libvoikko](https://github.com/voikko/corevoikko): Voikko-ohjelmiston avoin kirjasto suomen kielen käsittelyyn
- [gensim](https://github.com/RaRe-Technologies/gensim): "Topic modeling for humans"
- [pyLDAvis](https://github.com/bmabey/pyLDAvis): LDA-algoritmin tulosten visualisointi

In [1]:
import itertools
import pgdb
import libvoikko
from tqdm import tqdm_notebook as tqdm
from gensim.models.phrases import Phrases, Phraser
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning) 

DATABASE = 'vauvafi'
POST_QUERY = 'SELECT id, content FROM posts'
INSERT_NORMALIZED = 'INSERT INTO normalized_posts(id, content) VALUES(%s, %s)'
NORMALIZED_POST_QUERY = 'SELECT * FROM normalized_posts'
NORMALIZED_TOPIC_QUERY = '''
    SELECT topic_id as id, array_concat_agg(normalized_posts.content) AS content
    FROM posts LEFT JOIN normalized_posts
    ON posts.id = normalized_posts.id
    GROUP BY topic_id
    '''

VOIKKO = libvoikko.Voikko('fi')
BATCH_SIZE = 100

with open('stop_words.txt') as f:
    stop_words = [word.rstrip() for word in f]
    
with pgdb.connect(database=DATABASE) as db:
    with db.cursor() as cursor:
        cursor.execute('SELECT COUNT(id) FROM posts')
        POST_COUNT = cursor.fetchone().count

## Esikäsittely
* ylimääräisten välimerkkien ja sanojen poistaminen
* sanojen perusmuotoistaminen (lemmaus)

In [2]:
def tokenize(text):
    return VOIKKO.tokens(text)

tokenize('Onpa kivaa, kun on perjantai!')

[<Onpa,WORD>,
 < ,WHITESPACE>,
 <kivaa,WORD>,
 <,,PUNCTUATION>,
 < ,WHITESPACE>,
 <kun,WORD>,
 < ,WHITESPACE>,
 <on,WORD>,
 < ,WHITESPACE>,
 <perjantai,WORD>,
 <!,PUNCTUATION>]

In [3]:
def lemmatize(word):
    analysis = VOIKKO.analyze(word)
    try:
        return analysis[0]['BASEFORM'].lower()
    except (IndexError, KeyError):
        pass
    
lemmatize('perjantainakinkohan')

'perjantai'

In [4]:
def is_word(token):
    return token.tokenType == libvoikko.Token.WORD

def is_stop_word(word):
    return word in stop_words

def normalize(text):
    words = []
    for token in tokenize(text):
        if not is_word(token):
            continue
        word = lemmatize(token.tokenText)
        if word and not is_stop_word(word):
            words.append(word)
    return words

normalize('Onpa kivaa, kun on perjantai!')

['kiva', 'perjantai']

* ilmauksien (usean sanan yhdistelmät) mallinnus (_phrase modeling_)
    * tavoitteena säilyttää merkitykselliset ilmaukset myöhemmin käytettävässä bag-of-words-esityksessä
    * palkka juosta => palkka\_juosta
    * [gensim](https://radimrehurek.com/gensim/models/phrases.html): yksinkertainen sanojen esiintymistiheyksiin perustuva toteutus

In [5]:
def get_rows(query):
    with pgdb.connect(database=DATABASE) as db:
        with db.cursor() as cursor:
            cursor.execute(query)
            while True:
                rows = cursor.fetchmany(BATCH_SIZE)
                if not rows:
                    break
                yield from rows
            
def get_posts():
    yield from tqdm(get_rows(POST_QUERY), total=POST_COUNT)
    
def split_to_sentences(text):
    return VOIKKO.sentences(text)

def get_normalized_sentences():
    for post in get_posts():
        for sentence in split_to_sentences(post.content):
            yield normalize(sentence.sentenceText)
            
list(itertools.islice(get_normalized_sentences(), 50, 51))

HBox(children=(IntProgress(value=0, max=252738), HTML(value='')))

[['lapsi', 'hankkia', 'hoitaa', 'tuntua', 'vastenmielinen']]

In [6]:
def build_phrase_model(sentences):
    return Phraser(Phrases(sentences))

def pre_process(text, bigram, trigram):
    return list(trigram[bigram[normalize(text)]])

bigram = build_phrase_model(get_normalized_sentences())
trigram = build_phrase_model(bigram[get_normalized_sentences()])

HBox(children=(IntProgress(value=0, max=252738), HTML(value='')))

HBox(children=(IntProgress(value=0, max=252738), HTML(value='')))

In [7]:
' '.join(
    pre_process(
        '''
        Ammuin ilmapistoolilla koulukaveriani jalkaan.
        Sillä oli tiukat farkut jalassa, ei mennyt läpi niistäkään, mutta reiteen tuli mustelma.
        Elettiin kasarin alkua.
        ''',
        bigram,
        trigram
    )
)

'ampua ilmapistooli koulukaveri jalka tiukka_farkku jalka menty reisi mustelma elää kasari alku'

In [8]:
with pgdb.connect(database=DATABASE) as db:
    with db.cursor() as cursor:
        cursor.executemany(INSERT_NORMALIZED, (
            (post.id, pre_process(post.content, bigram, trigram))
            for post in get_posts()
        ))
    db.commit()

HBox(children=(IntProgress(value=0, max=252738), HTML(value='')))

In [9]:
def get_normalized_posts():
    for post in tqdm(get_rows(NORMALIZED_POST_QUERY), total=POST_COUNT):
        words = [word for word in post.content if len(word) > 2]
        if len(words) > 2:
            yield words

## Latent Dirichlet Allocation (LDA)
* [Blei, David M., Andrew Y. Ng, and Michael I. Jordan. 2003.](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf)
* dokumentit (_document_) muodostuvat useasta aiheesta (_topic_), jotka koostuvat useasta sanasta (_word_)
* mallin parametrit estimoidaan havaituista dokumenteista
* algoritmi on täysin ohjaamaton (_unsupervised_)

In [10]:
def build_dictionary():
    dictionary = Dictionary(get_normalized_posts())
    dictionary.filter_extremes(no_below=20, no_above=0.2)
    dictionary.compactify()
    return dictionary

dictionary = build_dictionary()

HBox(children=(IntProgress(value=0, max=252738), HTML(value='')))

In [11]:
def get_post_bows(dictionary):
    for post in get_normalized_posts():
        yield dictionary.doc2bow(post)

def build_corpus(dictionary, fname):
    bow_generator = get_post_bows(dictionary)
    MmCorpus.serialize(fname, bow_generator)
    corpus = MmCorpus(fname)
    return corpus

corpus = build_corpus(dictionary, 'vauvafi_corpus.mm')

HBox(children=(IntProgress(value=0, max=252738), HTML(value='')))

In [12]:
%%time

lda = LdaMulticore(corpus, num_topics=10, id2word=dictionary, workers=3)

CPU times: user 1min 13s, sys: 9.38 s, total: 1min 23s
Wall time: 1min 23s


In [13]:
ldavis_data = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(ldavis_data)

## Yhteenveto

* esikäsittelyllä suuri merkitys lopputuloksen kannalta
* suomi on vaikeaa, huono suomi vielä vaikeampaa
    * lemmaus/stemmaus, sanojen monimerkityksellisyys
    * valmiita työkaluja/kirjastoja huonommin saatavilla (vastineita Voikolle esim. [nltk](http://www.nltk.org/) ja [spaCy](https://spacy.io/))
* pidemmälle ajanjaksolle dynaaminen aihemallinnus (ajan myötä muuttuvat aihejakaumat)

* mitä hyötyä tästä sitten oli?
    * visualisointi
    * ymmärrys laajan dokumenttikokonaisuuden aiheista on usein itsessään hyödyllinen (huom. aiheisiin ei kuitenkaan _varsinaisesti_ liity merkitystä)
    * dokumenttien aihejakaumia voidaan hyödyntää bag-of-words-vektoreita pienempiulotteisena esityksinä jonkin muun mallin opetuksessa (esim. luokittelu- ja regressio-ongelmat)
    * aihemallinnusta sovellettu myös esim. geneettiseen dataan, kuviin ja sosiaalisiin verkostoihin ([David M. Blei. 2012.](http://www.cs.columbia.edu/~blei/papers/Blei2012.pdf))